In [1]:
import os
import time
import re
import uuid
import logging

from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By


load_dotenv()

True

In [2]:
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')
TIME_GAP_BETWEEN_ACTIONS = 5

In [3]:
class Logger:
    def __init__(self, filename: str, level: int=logging.INFO) -> None:
        self.filename = filename
        self.level = level

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

        file_handler = logging.FileHandler(self.filename)
        file_handler.setFormatter(formatter)

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(self.level)
        self.logger.addHandler(file_handler)

    def log(self, message: str, level: int=logging.INFO) -> None:
        if level == logging.INFO:
            self.logger.info(message)
        elif level == logging.WARNING:
            self.logger.warning(message)
        elif level == logging.ERROR:
            self.logger.error(message)
        else:
            self.logger.debug(message)

    def info(self, message):
        self.log(message, logging.INFO)

    def warning(self, message):
        self.log(message, logging.WARNING)

    def error(self, message):
        self.log(message, logging.ERROR)


In [4]:
class InstagramBot:
    def __init__(self, username: str, password: str, time_gap_between_actions: int = 5) -> None:
        self.id = uuid.uuid4()

        self.username = username
        self.password = password

        self.time_gap_between_actions = time_gap_between_actions

        self.base_url = "https://www.instagram.com/"

        self.driver =  webdriver.Chrome()

        self.logger = Logger(f"logs/{self.id}")

    def login(self) -> None:
        self.logger.info("login - Logging in...")

        self.logger.info("login - Going to Instagram...")
        self.driver.get(self.base_url) 
        self.__sleep()

        self.logger.info("login - Entering username...")
        self.driver\
            .find_element("xpath", "//input[@name=\"username\"]")\
            .send_keys(self.username)
        self.__sleep()
        
        self.logger.info("login - Entering password...")
        self.driver\
            .find_element("xpath", "//input[@name=\"password\"]")\
            .send_keys(self.password)
        self.__sleep()

        self.logger.info("login - Submitting...")
        self.driver\
            .find_element("xpath", '//button[@type="submit"]')\
            .click()
        self.__sleep()

        self.logger.info("login - Not now...")
        self.driver\
            .find_element("xpath", "//div[contains(text(), 'Not now')]")\
            .click()
        self.__sleep()

        self.logger.info("login - Not Now...")
        self.driver\
            .find_element("xpath", "//button[contains(text(), 'Not Now')]")\
            .click()
        self.__sleep()

        self.logger.info("login - Logged in.")

    def fetch_posts(self, num_scrolls: int) -> list:
        self.logger.info("posts - Fetching posts...")

        self.driver.get("https://www.instagram.com/explore/")

        res = []
        for _ in range(num_scrolls):
            self.logger.info("posts - Scrolling...")

            raw_posts = self.driver\
                .find_elements(By.TAG_NAME, "a")
            self.__sleep()
            
            posts = []
            for x in raw_posts:
                try:
                    if ".com/p" in x.get_attribute("href"):
                        posts.append(x.get_attribute("href"))
                except Exception as _:
                    continue

            self.logger.info(f"posts - Fetched {len(posts)} posts...")

            res += posts

            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            self.__sleep()
        res = list(set(res))

        self.logger.info(f"posts - Fetched {len(res)} unique posts.")

        return res
    
    def fetch_reels(self, num_iters: int):
        self.logger.info("reels - Fetching reels...")

        res = []
        for _ in range(num_iters):
            self.driver.get("https://www.instagram.com/reels/")
            self.__sleep()

            raw_reels = self.driver\
                .find_elements(By.TAG_NAME, "a")
            
            for x in raw_reels:
                try:
                    pattern = r'https://www\.instagram\.com/reels/[a-zA-Z0-9_-]+/#'

                    matches = re.findall(pattern, x.get_attribute("href"))

                    for match in matches:
                        res.append(match)
                        break
                except Exception as _:
                    continue
        res = list(set(res))

        self.logger.info(f"reels - Fetched {len(res)} unique reels.")

        return res
    
    def engage_posts(self, post_urls: list) -> None:
        for post_url in post_urls:
            try:
                self.engage_post(post_url)
            except Exception as _:
                continue

    def disengage_posts(self, post_urls: list) -> None:
        for post_url in post_urls:
            try:
                self.disengage_post(post_url)
            except Exception as _:
                continue

    def engage_reels(self, reel_urls: list) -> None:
        for reel_url in reel_urls:
            try:
                self.engage_reel(reel_url)
            except Exception as _:
                continue

    def disengage_reels(self, reel_urls: list) -> None:
        for reel_url in reel_urls:
            try:
                self.disengage_reel(reel_url)
            except Exception as _:
                continue
    
    def engage_post(self, post_url: str) -> None:
        self.logger.info(f"posts - Engaging with post {post_url}...")

        self.driver.get(post_url)
        self.__sleep()

        self.driver\
            .find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[3]/div[1]/div[1]/span[1]/div/div")\
            .click()
        self.__sleep()
        
        self.driver\
            .find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div[2]/div")\
            .click()
        self.__sleep()

        self.logger.info(f"posts - Enagaded with post {post_url}.")

    def disengage_post(self, post_url: str) -> None:
        self.logger.info(f"posts - Disengaging with post {post_url}...")

        self.driver.get(post_url)
        self.__sleep()

        self.driver\
            .find_element("xpath", '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[3]/div[1]/div[1]/span[1]/div/div')\
            .click()
        self.__sleep()
        
        self.driver\
            .find_element("xpath", '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[1]/div/div[2]/div/div[1]/div[2]/div')\
            .click()
        self.__sleep()

        self.driver\
            .find_element("xpath", "//button[contains(text(), 'Unfollow')]")\
            .click()
        self.__sleep()

        self.logger.info(f"posts - Disenagaded with post {post_url}.")

    def engage_reel(self, reel_url: str) -> None:
        self.logger.info(f"reels - Engaging with reel {reel_url}...")

        self.driver.get(reel_url)
        self.__sleep()

        self.driver\
            .find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div[1]/span")\
            .click()
        self.__sleep()
        
        self.driver\
            .find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div[1]/div[2]/div/div[1]/div/div")\
            .click()
        self.__sleep()

        self.logger.info(f"reels - Engaged with reel {reel_url}.")

    def disengage_reel(self, reel_url: str) -> None:
        self.logger.info(f"reels - Disengaging with reel {reel_url}...")

        self.driver.get(reel_url)
        self.__sleep()

        self.driver\
            .find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div[1]/span")\
            .click()
        self.__sleep()
        
        self.driver\
            .find_element("xpath", "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div/div/div/div[1]/div[2]/div/div[1]/div/div")\
            .click()
        self.__sleep()

        self.logger.info(f"reels - Disengaged with reel - {reel_url}.")

    def __sleep(self):
        time.sleep(self.time_gap_between_actions)

In [5]:
bot = InstagramBot(USERNAME, PASSWORD, TIME_GAP_BETWEEN_ACTIONS)

In [6]:
bot.login()

In [7]:
posts = bot.fetch_posts(2)

In [8]:
bot.engage_posts(posts[:2])

In [9]:
bot.disengage_posts(posts[:2])

In [10]:
reels = bot.fetch_reels(5)

In [11]:
bot.engage_reels(reels)

In [12]:
bot.disengage_reels(reels)